In [1]:
__author__ = 'Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20240124' # yyyymmdd; version datestamp of this notebook
#__datasets__ = ['allwise','smash']
__keywords__ = ['flamingos-2','gemini','browndwarf','dwarf','dragons']

# Gemini Flamingos-2 reduction using DRAGONS Python API
***
## Public archival data from f2img_tutorial - GS-2013B-Q-15 (WISE J041358.14-475039.3)
#### adapted from https://dragons.readthedocs.io/projects/f2img-drtutorial/en/v3.1.0/01_introduction.html
#### don't forget to `conda install -n dragons nb_conda_kernels ipykernel` to run this notebook on the DRAGONS env 
***

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create File Lists](#File_Lists)
* [Create Master Dark](#Master_dark)
* [Create Bad Pixel Mask](#BPM)
* [Create Master Flat Field](#Master_Flat)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to perform Flamingos-2 imaging data reduction using the Gemini DRAGONS package on the Data Lab science platform. Uses a custom DRAGONS kernel `"DRAGONS (Py3.7)"`. The steps include downloading data from the Gemini archive, setting up a DRAGONS calibration service, processing of flats, darks, bad pixel mask, and science frames, and finally the creation of a single combined stacked image.

<a class="anchor" id="summary"></a>
# Summary
DRAGONS is a Python-based astronomical data reduction platform written by the Gemini Science User Support Department. It currently can be used to reduce imaging data from Gemini instruments GMOS, NIRI, Flamingos 2, GSAOI, and GNIRS, and spectroscopic data in GMOS longslit mode. Linked here is a general list of guides, manuals, and tutorials about the use of DRAGONS:
https://dragons.readthedocs.io/en/v3.1.0/

The DRAGONS kernel has been made available in the Data Lab environment, which should allow users to access the routines without being dependent on installing the software in their local machines. 

In this notebook, we present an example of a DRAGONS Jupyter notebook that works in the Data Lab environment to fully reduce example Gemini South Flamingos-2 Y-band imaging data.
This is a version of the DRAGONS Jupyter notebook tutorial presented here: 
https://dragons.readthedocs.io/projects/f2img-drtutorial/en/v3.1.0/01_introduction.html

This notebook will not present all of the details of the many options available to adjust or optimize the DRAGONS Flamingos-2 data reduction process, rather will just show one example of a standard reduction of a Flamingos-2 imaging dataset. 
More extensive explanations can be found in the general DRAGONS GMOS data reduction tutorial from Gemini linked here:
https://dragons.readthedocs.io/projects/f2img-drtutorial/en/v3.1.0/index.html

The data used in this notebook example is Flamingos-2 Y band imaging from the Gemini archive of the brown dwarf W0413-4750 from the Gemini South program "A Study of the 450K Transition from T to Y Dwarf, and of the 350K Y Dwarfs", PI: S. Leggett, program ID GS-2013B-Q-15. More program information is given here: https://archive.gemini.edu/programinfo/GS-2013B-Q-15.

<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

* DRAGONS publication: Labrie et al., "DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South", ASPC, 523, 321L, https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract

* DRAGONS open source software publication: https://zenodo.org/record/7776065#.ZDg5qOzMLUI


<a class="anchor" id="imports"></a>
# Importing Python Libraries** (you'll probably have to install the `wget` and `ipympl` libraries)

In [ ]:
from __future__ import print_function

import glob
import wget

from gempy.adlibrary import dataselect
from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce
from gempy.utils import logutils

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize
from matplotlib.colors import PowerNorm

from collections import namedtuple

#%matplotlib widget
%matplotlib inline

<a class="anchor" id="About"></a>
# About the dataset
This data comes from an Flamingos-2 observation program that observced a star and a distant galaxy field with dither on target for sky subtraction. 

The calibrations we use in this example include:

- Darks for the science frames.
- Flats, as a sequence of lamps-on and lamps-off exposures.
- Short darks to use with the flats to create a bad pixel mask.

| Observation Type | File name(s) | Purpose and Exposure (seconds) |
| :--- | :--- | :---: |
| Science | S20131121S0075-083 | Y-band, 120 s |
| Darks | S20131121S0369-375 | 2 s, short darks for BPM |
| Darks | S20131120S0115-120 <br> S20131121S0010 <br> S20131122S0012 <br> S20131122S0438-439| 120 s, for science data <br><br><br><br>|
| Flats | S20131129S0320-323 | 20 s, Lamp On, Y-band |
| Flats | S20131126S1111-116 | 20 s, Lamp Off, Y-band |

<a class="anchor" id="Downloading_Data"></a>
# Downloading the data (direct link to .tar file hosted at Gemini - 125 Megabytes)

In [ ]:
# uncomment the line and run the cell
#
#wget.download("http://www.gemini.edu/sciops/data/software/datapkgs/f2img_tutorial_datapkg-v1.tar")

**Downloading the data** (individual files - direct link to the Gemini Archive - 577 Megabytes)

In [ ]:
# # uncomment the lines and run the cell

# # Science 

#wget.download("http://archive.gemini.edu/file/S20131121S0075.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0076.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0077.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0078.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0079.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0080.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0081.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0082.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0083.fits")

# # Darks (2 s, short darks for BPM)

#wget.download("http://archive.gemini.edu/file/S20131121S0369.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0370.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0371.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0372.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0373.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0374.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0375.fits")

# # Darks (120 s, for science data)

#wget.download("http://archive.gemini.edu/file/S20131120S0115.fits")
#wget.download("http://archive.gemini.edu/file/S20131120S0116.fits")
#wget.download("http://archive.gemini.edu/file/S20131120S0117.fits")
#wget.download("http://archive.gemini.edu/file/S20131120S0118.fits")
#wget.download("http://archive.gemini.edu/file/S20131120S0119.fits")
#wget.download("http://archive.gemini.edu/file/S20131120S0120.fits")
#wget.download("http://archive.gemini.edu/file/S20131121S0010.fits")
#wget.download("http://archive.gemini.edu/file/S20131122S0012.fits")
#wget.download("http://archive.gemini.edu/file/S20131122S0438.fits")
#wget.download("http://archive.gemini.edu/file/S20131122S0439.fits")

# # Flats (20 s, Lamp On, Y-band)

#wget.download("http://archive.gemini.edu/file/S20131129S0320.fits")
#wget.download("http://archive.gemini.edu/file/S20131129S0321.fits")
#wget.download("http://archive.gemini.edu/file/S20131129S0322.fits")
#wget.download("http://archive.gemini.edu/file/S20131129S0323.fits")

# # Flats (20 s, Lamp Off, Y-band)

#wget.download("http://archive.gemini.edu/file/S20131126S1111.fits")
#wget.download("http://archive.gemini.edu/file/S20131126S1112.fits")
#wget.download("http://archive.gemini.edu/file/S20131126S1113.fits")
#wget.download("http://archive.gemini.edu/file/S20131126S1114.fits")
#wget.download("http://archive.gemini.edu/file/S20131126S1115.fits")
#wget.download("http://archive.gemini.edu/file/S20131126S1116.fits")

# Create and move data to raw/ directory (uncomment first)

In [ ]:
#!mkdir raw/
#!mv S2013*.fits raw/

In [ ]:
# Check header of one raw science image
#
tmp = fits.open("raw/S20131121S0075.fits")
tmp[0].header

<a class="anchor" id="DRAGONS_logger"></a>
# Setting up the DRAGONS logger

DRAGONS comes with a local calibration manager that uses the same calibration association rules as the Gemini Observatory Archive. This allows reduce to make requests to a local light-weight database for matching processed calibrations when needed to reduce a dataset.

This simply tells the system where to put the calibration database, the database that will keep track of the processed calibrations we are going to send to it.

In [ ]:
logutils.config(file_name='f2_data_reduction.log')

**Create a list of all the FITS files in the directory**

In [ ]:
all_files = glob.glob('raw/S2013*[0-9].fits')
all_files.sort()
#all_files

<a class="anchor" id="File_Lists"></a>
# Create file lists

This data set contains science and calibration frames. For some programs, it could have different observed targets and different exposure times depending on how you like to organize your raw data.

The DRAGONS data reduction pipeline does not organize the data for you. You have to do it. DRAGONS provides tools to help you with that.

The first step is to create lists that will be used in the data reduction process. For that, we use dataselect. Please, refer to the dataselect documentation for details regarding its usage.

In [ ]:
dark_files_120s = dataselect.select_data(
    all_files,['F2', 'DARK', 'RAW'],[],
    dataselect.expr_parser('exposure_time==120'))

dark_files_2s = dataselect.select_data(
    all_files,['F2', 'DARK', 'RAW'],[],
    dataselect.expr_parser('exposure_time==2'))

list_of_flats_Y = dataselect.select_data(
     all_files,['FLAT'],[],
     dataselect.expr_parser('filter_name=="Y"'))

list_of_science_images = dataselect.select_data(
    all_files,['F2'],[],
    dataselect.expr_parser('(observation_class=="science" and filter_name=="Y")'))

<a class="anchor" id="Master_dark"></a>
# Create Master Dark 

In [ ]:
reduce_darks = Reduce()
reduce_darks.files.extend(dark_files_120s)
reduce_darks.runr()

<a class="anchor" id="BPM"></a>
# Create a Bad Pixel Mask

The Bad Pixel Mask (BPM) can be built using a set of flat images with the lamps on and off and a set of short exposure dark files. Here, our shortest dark files have 2 second exposure time. Again, we use the reduce command to produce the BPMs.

It is important to note that the recipe library association is done based on the nature of the first file in the input list. Since the recipe to make the BPM is located in the recipe library for flats, the first item in the list must be a flat.

For Flamingos-2, the filter wheel’s location is such that the choice of filter does not interfere with the results. Here we have Y-band flats, so we will use Y-band flats.

In [ ]:
reduce_bpm = Reduce()
reduce_bpm.files.extend(list_of_flats_Y)
reduce_bpm.files.extend(dark_files_2s)
reduce_bpm.recipename = 'makeProcessedBPM'
reduce_bpm.runr()

bpm_filename = reduce_bpm.output_filenames[0]

<a class="anchor" id="Master_Flat"></a>
# Create a Master Flat Field

The F2 Y-band master flat is created from a series of lamp-on and lamp-off exposures. They should all have the same exposure time. Each flavor is stacked (averaged), then the lamp-off stack is subtracted from the lamp-on stack and the result normalized.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(list_of_flats_Y)
reduce_flats.uparms = [('addDQ:user_bpm', bpm_filename)]
reduce_flats.runr()

<a class="anchor" id="Reduce_Science"></a>
# Reduce the Science Images
This command retrieves the master dark and the master flat, and applies them to the science data. For sky subtraction, the software analyses the sequence to establish whether this is a dither-on-target or an offset-to-sky sequence and then proceeds accordingly. Finally, the sky-subtracted frames are aligned and stacked together. Sources in the frames are used for the alignment.

The final product file will have a _stack.fits suffix.

The output stack units are in electrons (header keyword BUNIT=electrons). The output stack is stored in a multi-extension FITS (MEF) file. The science signal is in the “SCI” extension, the variance is in the “VAR” extension, and the data quality plane (mask) is in the “DQ” extension.

In [ ]:
reduce_target = Reduce()
reduce_target.files.extend(list_of_science_images)
reduce_target.uparms = [('addDQ:user_bpm', bpm_filename)]
reduce_target.runr()

<a class="anchor" id="Display_Image"></a>
# Display the Stacked Image

In [ ]:
image_file = "S20131121S0075_stack.fits"
hdu_list = fits.open(image_file)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (15,15))
plt.imshow(image_data,cmap='bone',norm=Normalize(vmin=1, vmax=1000),origin='lower')
plt.show()

<a class="anchor" id="Clean-up"></a>
# Optional: remove duplicate calibrations and remove raw data (uncomment lines before running)

In [ ]:
#!rm -rf raw/